In [9]:
import tensorflow as tf
import numpy as np

import aboleth as ab

In [10]:
tf.logging.set_verbosity(tf.logging.INFO)

rseed = 100
ab.set_hyperseed(rseed)

# Optimization
n_epochs = 25
batch_size = 100
config = tf.ConfigProto(device_count={'GPU': 1})  # Use GPU ?

reg = 1e-1

l_samples = 1

In [11]:
# Network architecture
net = ab.stack(
    ab.InputLayer(name='X', n_samples=l_samples),  # LSAMPLES,BATCH_SIZE,28*28
    ab.Reshape(target_shape=(28, 28, 1)),  # LSAMPLES, BATCH_SIZE, 28, 28, 1
    
    ab.Conv2DMAP(filters=32,
                 kernel_size=(3, 3),
                 l1_reg=0., l2_reg=reg),  # LSAMPLES, BATCH_SIZE, 28, 28, 32
    ab.Activation(h=tf.nn.relu),
    ab.Conv2DMAP(filters=32,
                 kernel_size=(3, 3),
                 l1_reg=0., l2_reg=reg),  # LSAMPLES, BATCH_SIZE, 28, 28, 32
    ab.Activation(h=tf.nn.relu),
    ab.MaxPool2D(pool_size=(2, 2),
                 strides=(2, 2)),  # LSAMPLES, BATCH_SIZE, 14, 14, 32    
    ab.DropOut(0.25),

    ab.Conv2DMAP(filters=64,
                 kernel_size=(3, 3),
                 l1_reg=0., l2_reg=reg),  # LSAMPLES, BATCH_SIZE, 28, 28, 32
    ab.Activation(h=tf.nn.relu),
    ab.Conv2DMAP(filters=64,
                 kernel_size=(3, 3),
                 l1_reg=0., l2_reg=reg),  # LSAMPLES, BATCH_SIZE, 28, 28, 32
    ab.Activation(h=tf.nn.relu),
    ab.MaxPool2D(pool_size=(2, 2),
                 strides=(2, 2)),  # LSAMPLES, BATCH_SIZE, 14, 14, 32    
    ab.DropOut(0.25),
    
    ab.Reshape(target_shape=(7*7*64,)),  # LSAMPLES, BATCH_SIZE, 7*7*64
    ab.DenseMAP(output_dim=512,
                l1_reg=0., l2_reg=reg),  # LSAMPLES, BATCH_SIZE, 1024
    ab.Activation(h=tf.nn.relu),
    ab.DropOut(0.5),

    ab.DenseMAP(output_dim=10,
                l1_reg=0., l2_reg=reg),  # LSAMPLES, BATCH_SIZE, 10
)

In [12]:
# Dataset
mnist_data = tf.contrib.learn.datasets.mnist.read_data_sets(
    './mnist_demo', reshape=True)

N, D = mnist_data.train.images.shape

X, Y = tf.data.Dataset.from_tensor_slices(
    (np.asarray(mnist_data.train.images, dtype=np.float32),
     np.asarray(mnist_data.train.labels, dtype=np.int64))
).repeat(n_epochs).shuffle(N).batch(batch_size) \
 .make_one_shot_iterator().get_next()

# Xs, Ys = tf.data.Dataset.from_tensor_slices(
#     (np.asarray(mnist_data.test.images, dtype=np.float32),
#      np.asarray(mnist_data.test.labels, dtype=np.int64))
# ).repeat(n_epochs).make_one_shot_iterator().get_next()

Xs = np.asarray(mnist_data.test.images, dtype=np.float32)
Ys = np.asarray(mnist_data.test.labels, dtype=np.int64)

Extracting ./mnist_demo/train-images-idx3-ubyte.gz
Extracting ./mnist_demo/train-labels-idx1-ubyte.gz
Extracting ./mnist_demo/t10k-images-idx3-ubyte.gz
Extracting ./mnist_demo/t10k-labels-idx1-ubyte.gz


In [13]:
# Model specification
with tf.name_scope("model"):
    logits, reg = net(X=X)
    llh = tf.distributions.Categorical(logits=logits)
    loss = ab.elbo(llh, Y, N, reg)
    #loss = ab.max_posterior(llh, Y, reg)
    probs = ab.sample_mean(llh.probs)
    accuracy = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(probs, axis=1), Y), dtype=tf.float32))

In [14]:
# Training graph building
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-1)
    global_step = tf.train.get_or_create_global_step()
    train = optimizer.minimize(loss, global_step=global_step)

In [15]:
logger = tf.train.LoggingTensorHook(
    dict(step=global_step, loss=loss, accuracy=accuracy),
    every_n_iter=500
)

In [16]:
with tf.train.MonitoredTrainingSession(
    config=config,
    hooks=[logger]
) as sess:

    while not sess.should_stop():

        step, _ = sess.run([global_step, train])

        # this part is a bit superfluous. Could get it working with
        # Datasets too but just wanted to sanity test for now.
        if not step % 500:
            val_acc = accuracy.eval(feed_dict={X: Xs, Y: Ys}, session=sess)
            tf.logging.info("step = %d, validation accuracy: %f",
                            step, val_acc)

FailedPreconditionError: Attempting to use uninitialized value model_1/b_map_2
	 [[Node: model_1/b_map_2/read = Identity[T=DT_FLOAT, _class=["loc:@model_1/b_map_2"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](model_1/b_map_2)]]

Caused by op 'model_1/b_map_2/read', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python3.5/